In [54]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
import pandas as pd
import numpy as np
import bs4
import lxml.etree as xml
import time
import datetime
from tqdm import tqdm
import random

In [3]:
PATH = '/Users/Asus/Downloads/chromedriver' # Defines where my cromedriver package is installed. Opens an automated Chrome window.
driver = webdriver.Chrome(PATH) # names webdriver as driver and assigns path. 
driver.quit() # closes the driver so the window is not always on. 

In [34]:
url = 'https://www.bolha.com/nepremicnine/lokacija-stanovanja-studenci-90.00-m2-oglas-5298888'

In [148]:
driver = webdriver.Chrome(PATH) 
driver.get(url) # automatically opens Nepremicnine.net
# driver.quit() # this closes the window. 
html = driver.page_source
webPage = bs4.BeautifulSoup(html, "lxml")
driver.quit()

###  What can be scraped from each ad?
#### Nepremicnine ads: 
From more info: (4)
- type (stanovanje, hiša..)
- region (LJ-Mesto, Gorenjska...)
- upravna enota (Lj. Bežigrad..)
- municipality (LJubljana) 
From opis: 
kratek: (7)
- size(m2)
- class (3- sobno..)
- built (leto izgradnje) 
- renovated (leto adaptirano)
- position (2/10... nadstropje..)
- price
- price-old
slika: 
- dmiss (label Ne spreglejte -yes/no)
potential: 
- number of pictures( from: .../galerija/) 


#### Bolha ads: 
- price (from li class:'price-item price-item--base'>
ul class ='meta-items'>: (3)
- posted (date)
- lasts ( objavljen se.)
- t_shown (prikazov:)
tbody (tabela s podatki): 
 - price_type 




 


In [31]:
d_df = pd.DataFrame.from_dict([d])
d_df

,Vrsta cene,Posredovanje,Lokacija,Umeščenost stanovanja,Nadstropje,Vrsta stanovanja,Bivalna površina,Neto površina,Leto izgradnje*,Balkon / terasa / atrij,Interna številka oglasa
0,Skupna cena,Prodam,"Osrednjeslovenska, Ivančna Gorica, Ivančna Gorica",V večstanovanjski stavbi,Pritličje,3-sobno,"61,50 m²","66,00 m²",2005,Balkon,1052


In [36]:

a = {}
for item in webPage.find_all('tr'):
    f_names = item.find('th')
    f_atr = item.find('td')
    a[f_names.text.replace(':', '')] = f_atr.text
    #print(f_names.text)
    #print(f_atr.text)
print(a)
print('---------')

{'Vrsta cene': 'Skupna cena', 'Posredovanje': 'Prodam', 'Lokacija': 'Podravska, Maribor, Studenci', 'Umeščenost stanovanja': 'V večstanovanjski stavbi', 'Nadstropje': '2.', 'Skupno število nadstropij v stavbi': '3', 'Vrsta stanovanja': '4,5-sobno', 'Bivalna površina': '90,30 m²', 'Neto površina': '98,40 m²', 'Leto izgradnje*': '2020', 'Opremljenost in stanje': 'Delno opremljeno', 'Balkon / terasa / atrij': 'Balkon, Loža', 'Energetski razred': 'B2 (25 - 35 kWh/m2a)'}
---------


In [133]:
# adds two different DataFrames together and inputs nan where they do not overlap in columns :D 
# This discovery makes me very happy.
data = pd.concat([d_df,a_df], axis=0, ignore_index=True)
data

,Vrsta cene,Posredovanje,Lokacija,Umeščenost stanovanja,Nadstropje,Vrsta stanovanja,Bivalna površina,Neto površina,Leto izgradnje*,Balkon / terasa / atrij,Interna številka oglasa,Skupno število nadstropij v stavbi,Opremljenost in stanje,Energetski razred
0,Skupna cena,Prodam,"Osrednjeslovenska, Ivančna Gorica, Ivančna Gorica",V večstanovanjski stavbi,Pritličje,3-sobno,"61,50 m²","66,00 m²",2005,Balkon,1052,NaN,NaN,NaN
1,Skupna cena,Prodam,"Podravska, Maribor, Studenci",V večstanovanjski stavbi,2.,"4,5-sobno","90,30 m²","98,40 m²",2020,"Balkon, Loža",NaN,3,Delno opremljeno,B2 (25 - 35 kWh/m2a)


## Scrapes individual ads' content for info.

In [4]:
def scrape_content(page):
    a = {}
    
    # adding details about the ad (first published, times viewed...)
    ad_info = page.find('ul', class_ = 'meta-items')

    for item in ad_info.find_all('li'):   
        f_names = item.find('span', class_='label').text
        f_atr = item.find(class_='value').text

        a[f_names.replace(':', '')] = f_atr.replace(' ', '')

    # adding property price
    div_price = page.find('ul', class_ = 'price-items cf')
    f_names= div_price.find(class_='label').text
    f_price_raw =  div_price.find(class_='price price--hrk').text.replace(' ', '')
    
    f_atr = f_price_raw
    #f_price_raw if str(f_price_raw) == 'podogovoru' else f_price_raw.split('\n')[2].replace(u'\xa0', u' ')

    a[f_names.replace(':', '')] = f_atr

    # adding info about the property
    table_info = page.find('tbody')

    for item in table_info.find_all('tr'):
        f_names = item.find('th')
        f_atr = item.find('td')

        a[f_names.text.replace(':', '')] = f_atr.text
        
    # make dict a into a df. 
    a_df = pd.DataFrame.from_dict([a])
        
    return a_df


In [137]:
url_source = webPage

In [138]:
scrape_ads(url_source)

,Šifra oglasa,Objavljen,Do poteka še,Število prikazov oglasa,Cena,Vrsta cene,Posredovanje,Lokacija,Umeščenost stanovanja,Nadstropje,Skupno število nadstropij v stavbi,Vrsta stanovanja,Bivalna površina,Neto površina,Leto izgradnje*,Opremljenost in stanje,Balkon / terasa / atrij,Energetski razred
0,5298888,12.03.2021dne13:16,\ndoprodaje,2128-krat,"223.932,98 €",Skupna cena,Prodam,"Podravska, Maribor, Studenci",V večstanovanjski stavbi,2.,3,"4,5-sobno","90,30 m²","98,40 m²",2020,Delno opremljeno,"Balkon, Loža",B2 (25 - 35 kWh/m2a)


In [5]:
# adding df of every page to the main df.
def append_dfs(mainDF, newDF):
    df = pd.concat([mainDF,newDF], axis=0, ignore_index=True)
    return df

In [8]:
url_base = 'https://www.bolha.com/prodaja-stanovanja'

In [6]:
def getHTML_one(url):
    driver = webdriver.Chrome(PATH) 
    driver.get(url) # automatically opens Nepremicnine.net
    # driver.quit() # this closes the window. 
    html = driver.page_source
    driver.quit()
    webPage = bs4.BeautifulSoup(html, "lxml")
   
    return webPage

In [46]:
def getHTML(url):
    driver.get(url)
    html = driver.page_source
    webPage = bs4.BeautifulSoup(html, "lxml")
   
    return webPage

In [8]:
page = getHTML_one('https://www.bolha.com/prodaja-stanovanja?geo%5BlocationIds%5D=26325%2C26324%2C26323%2C26322%2C26321%2C26320%2C26319%2C26318%2C26317%2C26316%2C26315%2C26314%2C44285%2C30277&typeOfTransaction=sell&page=48')

In [8]:
def get_first_HTML(url):
    driver.get(url)
    html = driver.page_source
    webPage = bs4.BeautifulSoup(html, "lxml")
   
    return webPage

In [34]:
print([*data])

['Ad_ID', 'Ad_Link', 'datetime']


In [9]:
def ads_cutout(page):
    num = page.find('strong', class_="entities-count").text
    soup = page.find('div', class_='EntityList EntityList--Standard EntityList--Regular EntityList--ListItemRegularAd EntityList--itemCount_'+str(num)).prettify()
    ads = soup.find_all('h3', class_= 'entity-title')
    return ads

In [10]:
def scrape_links(page): 
    d = {'Ad_ID':[], 'Ad_Link':[], 'datetime' : []}
    num = page.find('strong', class_="entities-count").text
    soup = page.find('div', class_='EntityList EntityList--Standard EntityList--Regular EntityList--ListItemRegularAd EntityList--itemCount_'+str(num))
    ads = soup.find_all('h3', class_= 'entity-title')
    
    for ad in ads: 
        ad_link= ad.find('a', class_= 'link')['href']
        ad_id = ad_link.split('-')[-1]
        d['Ad_ID'].append(ad_id)
        d['Ad_Link'].append(ad_link)
        d['datetime'].append(datetime.datetime.now().strftime("%d.%m.%Y_%H:%M:%S"))
    #print(d)    
    d_df = pd.DataFrame(d,)
    print(d_df)
    return d_df

In [11]:
# url for all flats listed in Slovenia
url_b = 'https://www.bolha.com/prodaja-stanovanja?geo%5BlocationIds%5D=26325%2C26324%2C26323%2C26322%2C26321%2C26320%2C26319%2C26318%2C26317%2C26316%2C26315%2C26314%2C44285%2C30277&typeOfTransaction=sell&page=' 
# add page number at the end

In [12]:
def total_pages(page):
    t_ads = int(page.find('strong', class_='entities-count').text)
    t_pages = round(((t_ads / 25) +1))
    return t_pages

## Scraping all available ads and corresponding links. 

In [14]:
# working version

driver = webdriver.Chrome(PATH) 
base_url = url_b

page_base = get_first_HTML(base_url)
start = 1
end =  total_pages(page_base)
data = pd.DataFrame()


for i in tqdm(range(start,end +1)):
    urli = base_url + str(i)
    page = getHTML(urli)
    i_df = scrape_links(page)
    #print(i_df)
    data = pd.concat([data, i_df], axis=0, ignore_index=True)
              
driver.quit()

#data.to_csv("./bolha_flats_" + str(start) + '-'+ str(end) + '_' + str(date) + ".csv")

  2%|█▊                                                                                 | 1/47 [00:01<00:49,  1.08s/it]

      Ad_ID                                            Ad_Link  \
0   5784474  /nepremicnine/lokacija-stanovanja-center-125.0...   
1   6654426  /nepremicnine/prodamo-obnovljeno-1.5-sobno-sta...   
2   6624845  /nepremicnine/lokacija-stanovanja-kosaki-62.00...   
3   6624148  /nepremicnine/lokacija-stanovanja-kosaki-68.00...   
4   6617291  /nepremicnine/maribor-studenci-tabor-novogradn...   
5   6616383  /nepremicnine/lokacija-stanovanja-kosaki-77.00...   
6   6616201  /nepremicnine/lokacija-stanovanja-kosaki-128.0...   
7   6608652  /nepremicnine/maribor-kosaki-novogradnja-4-sob...   
8   3577634  /nepremicnine/maribor-studenci-106.00-m2-4-sob...   
9   3576924  /nepremicnine/maribor-studenci-104.00-m2-4-sob...   
10  3561472  /nepremicnine/maribor-studenci-104.00-m2-novog...   
11  3161746  /nepremicnine/lokacija-stanovanja-studenci-189...   
12  6654279  /nepremicnine/lokacija-stanovanja-lokrovec-ogl...   
13  1452831  /nepremicnine/stanovanje-pomurska-murska-sobot...   
14  665410

  4%|███▌                                                                               | 2/47 [00:02<00:46,  1.04s/it]

      Ad_ID                                            Ad_Link  \
0   6648004  /nepremicnine/lokacija-stanovanja-trzin-23.00-...   
1   6646987  /nepremicnine/lokacija-stanovanja-ravne-korosk...   
2   6307365  /nepremicnine/lokacija-stanovanja-center-oglas...   
3   1446157  /nepremicnine/stanovanje-podravska-sentilj-v-s...   
4   6201362  /nepremicnine/lokacija-stanovanja-skoke-90.00-...   
5   6644864  /nepremicnine/stanovanje-z-dvema-balkonoma-ob-...   
6   6634986  /nepremicnine/prodaja-stanovanje-trisobno-posa...   
7   3838389  /nepremicnine/lokacija-stanovanja-center-41.00...   
8   6142371  /nepremicnine/lokacija-stanovanja-ptuj-68.00-m...   
9   6261626  /nepremicnine/lokacija-stanovanja-sentilj-v-sl...   
10  6633614  /nepremicnine/stanovanje-smarje-pri-jelsah-49....   
11  1454333  /nepremicnine/apartmaj-hrvaska-murter-oglas-14...   
12  2439696  /nepremicnine/stanovanje-podravska-maribor-ost...   
13  6633204  /nepremicnine/lokacija-stanovanja-vas-69.00-m2...   
14  663219

  6%|█████▎                                                                             | 3/47 [00:03<00:45,  1.04s/it]

...

      Ad_ID                                            Ad_Link  \
0   4157318       /nepremicnine/stanovanje-opric-oglas-4157318   
1   4157315       /nepremicnine/stanovanje-selce-oglas-4157315   
2   4157309  /nepremicnine/stanovanje-vinodolski-oglas-4157309   
3   4157307         /nepremicnine/stanovanje-krk-oglas-4157307   
4   4155040  /nepremicnine/ljubljana-moste-stanovanje-1-sob...   
5   4155027  /nepremicnine/ljubljana-moste-stanovanje-3-sob...   
6   4155005  /nepremicnine/ljubljana-polje-stanovanje-3-sob...   
7   4154992  /nepremicnine/ljubljana-bezigrad-stanovanje-4-...   
8   4154983  /nepremicnine/ljubljana-zelena-jama-stanovanje...   
9   4154941  /nepremicnine/maribor-kosaki-stanovanje-ostalo...   
10  4149904  /nepremicnine/cudovito-mansardno-stanovanje-v-...   
11  4124412  /nepremicnine/stanovanje-vinodolski-oglas-4124412   
12  4108198  /nepremicnine/stanovanje-bled-presernova-cesta...   
13  4102274  /nepremicnine/prodaja-stanovanje-trisobno-podr...   
14  409054

 98%|████████████████████████████████████████████████████████████████████████████████▎ | 46/47 [00:49<00:01,  1.03s/it]

      Ad_ID                                            Ad_Link  \
0   4022106  /nepremicnine/prodaja-stanovanje-stirisobno-lj...   
1   4021936  /nepremicnine/prodaja-stanovanje-dvosobno-savi...   
2   4021934  /nepremicnine/prodaja-stanovanje-dvosobno-dva-...   
3   4021933  /nepremicnine/prodaja-stanovanje-dvosobno-savi...   
4   4021486  /nepremicnine/prodaja-stanovanje-dvosobno-podr...   
5   4021433  /nepremicnine/prodaja-stanovanje-enosobno-podr...   
6   4021324  /nepremicnine/prodaja-stanovanje-dupleks-savin...   
7   4021210     /nepremicnine/2-sobno-stanovanje-oglas-4021210   
8   4021204  /nepremicnine/stanovanjsko-poslovni-objekt-mar...   
9   4021189  /nepremicnine/3-sobno-stanovanje-v-starejsi-me...   
10  4021085  /nepremicnine/nedokoncan-vecstanovanjski-objek...   
11  4021043  /nepremicnine/dupleks-stanovanje-z-bazenom-sec...   
12  4020970  /nepremicnine/2-sobno-stanovanje-ruse-oglas-40...   
13  4020955  /nepremicnine/nedokoncana-stanovanjsko-poslovn...   
14  402095

100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [00:50<00:00,  1.07s/it]

     Ad_ID                                            Ad_Link  \
0  2218050  /nepremicnine/stanovanje-podravska-maribor-cen...   
1  2222020  /nepremicnine/stanovanje-gorenjska-cerklje-gor...   
2  2786494  /nepremicnine/lokacija-stanovanja-center-102.0...   
3  2223586  /nepremicnine/stanovanje-podravska-maribor-cen...   
4  2217657  /nepremicnine/stanovanje-pomurska-murska-sobot...   
5  2221094  /nepremicnine/stanovanje-pomurska-murska-sobot...   
6  2222326  /nepremicnine/stanovanje-osrednjeslovenska-lju...   
7  2218508  /nepremicnine/stanovanje-savinjska-zrece-3-sob...   

              datetime  
0  29.03.2021_12:41:05  
1  29.03.2021_12:41:05  
2  29.03.2021_12:41:05  
3  29.03.2021_12:41:05  
4  29.03.2021_12:41:05  
5  29.03.2021_12:41:05  
6  29.03.2021_12:41:05  
7  29.03.2021_12:41:05  


In [27]:
data

,Ad_ID,Ad_Link,datetime
0,5784474,/nepremicnine/lokacija-stanovanja-center-125.0...,29.03.2021_12:40:16
1,6654426,/nepremicnine/prodamo-obnovljeno-1.5-sobno-sta...,29.03.2021_12:40:16
2,6624845,/nepremicnine/lokacija-stanovanja-kosaki-62.00...,29.03.2021_12:40:16
3,6624148,/nepremicnine/lokacija-stanovanja-kosaki-68.00...,29.03.2021_12:40:16
4,6617291,/nepremicnine/maribor-studenci-tabor-novogradn...,29.03.2021_12:40:16
...,...,...,...
1153,2223586,/nepremicnine/stanovanje-podravska-maribor-cen...,29.03.2021_12:41:05
1154,2217657,/nepremicnine/stanovanje-pomurska-murska-sobot...,29.03.2021_12:41:05
1155,2221094,/nepremicnine/stanovanje-pomurska-murska-sobot...,29.03.2021_12:41:05
1156,2222326,/nepremicnine/stanovanje-osrednjeslovenska-lju...,29.03.2021_12:41:05


In [50]:
date = datetime.datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
data.to_csv("./bolha_flats_" + str(start) + '-'+ str(end) + '_' + str(date) + ".csv")

In [17]:
data1  = data[:50]
data1.head(5)

,Ad_ID,Ad_Link,datetime
0,5784474,/nepremicnine/lokacija-stanovanja-center-125.0...,29.03.2021_12:40:16
1,6654426,/nepremicnine/prodamo-obnovljeno-1.5-sobno-sta...,29.03.2021_12:40:16
2,6624845,/nepremicnine/lokacija-stanovanja-kosaki-62.00...,29.03.2021_12:40:16
3,6624148,/nepremicnine/lokacija-stanovanja-kosaki-68.00...,29.03.2021_12:40:16
4,6617291,/nepremicnine/maribor-studenci-tabor-novogradn...,29.03.2021_12:40:16


## Scraping individual ads into a DataFrame

In [56]:
# just added the function def, not tested yet. Integrate per-partes scraping for up to given limit (default = 100?)

def scraping_ads(df):
    driver = webdriver.Chrome(PATH) 
    data_ads = pd.DataFrame()
    base_url = 'https://www.bolha.com'

    for i in tqdm(range(0,len(df))):
        urli = base_url+str(df['Ad_Link'][i])
        page = getHTML(urli)
        i_df = scrape_content(page)

        data_ads = pd.concat([data_ads, i_df], axis=0, ignore_index=True)
        sleep_time = abs(np.random.normal(0.3,0.1))
        time.sleep(sleep_time)

    driver.quit()
    return data_ads

In [57]:
data_ads = scraping_ads(data)

100%|██████████████████████████████████████████████████████████████████████████████| 1158/1158 [20:22<00:00,  1.06s/it]


In [58]:
date = datetime.datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
data_ads.to_csv("./bolha_flats_all_info" + str(start) + '-'+ str(end) + '_' + str(date) + ".csv")

In [23]:
data['Ad_Link'][0]

'/nepremicnine/lokacija-stanovanja-center-125.00-m2-oglas-5784474'

In [94]:
data_ads['Cena']

0       \n\n192.000 €\n
1       \n\n134.500 €\n
2        \n\n29.500 €\n
3       \n\n120.000 €\n
4       \n\n180.000 €\n
             ...       
1183     \n\n60.000 €\n
1184    \n\n164.250 €\n
1185    \n\n150.015 €\n
1186    \n\n170.000 €\n
1187     \n\n90.000 €\n
Name: Cena, Length: 1188, dtype: object

In [4]:
data = pd.read_csv('bolha_flats_all_info1-49_14-03-2021_20-55-58.csv')
data.shape

(1188, 24)

In [6]:
data.head()

,Unnamed: 0,Šifra oglasa,Objavljen,Do poteka še,Število prikazov oglasa,Cena,Vrsta cene,Posredovanje,Lokacija,Umeščenost stanovanja,...,Neto površina,Leto izgradnje*,Leto zadnje prenove,Opremljenost in stanje,Energetski razred,Možnost menjave,Število parkirnih mest,Balkon / terasa / atrij,Zunanje površine in oprema,Interna številka oglasa
0,0,6523968,14.03.2021dne18:49,"\n29dni,22h,33min.\n",57-krat,\n\n192.000 €\n,Skupna cena,Prodam,"Savinjska, Celje, Slavko Šlander",V večstanovanjski stavbi,...,"105,00 m²",2004.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,6523894,14.03.2021dne18:39,"\n29dni,22h,24min.\n",29-krat,\n\n134.500 €\n,Skupna cena,Prodam,"Osrednjeslovenska, Ljubljana Center, Center",V večstanovanjski stavbi,...,"29,30 m²",1908.0,2020.0,Popolnoma opremljeno,EI ni potrebna (334. člen EZ-1),Možna menjava za drugo nepremičnino,NaN,NaN,NaN,NaN
2,2,6523589,14.03.2021dne18:07,"\n29dni,21h,52min.\n",33-krat,\n\n29.500 €\n,Skupna cena,Prodam,"Gorenjska, Tržič, Tržič",V večstanovanjski stavbi,...,"11,00 m²",1961.0,NaN,NaN,E (105 - 150 kWh/m2a),NaN,NaN,NaN,NaN,NaN
3,3,6523431,14.03.2021dne17:47,"\n29dni,21h,32min.\n",34-krat,\n\n120.000 €\n,Skupna cena,Prodam,"Savinjska, Slovenske Konjice, Slovenske Konjice",V večstanovanjski stavbi,...,"70,00 m²",2020.0,NaN,Neopremljeno,A2 (10 - 15 kWh/m2a),NaN,7+,"Balkon, Loža",NaN,NaN
4,4,6523190,14.03.2021dne17:16,"\n29dni,21h,0min.\n",23-krat,\n\n180.000 €\n,Skupna cena,Prodam,"Osrednjeslovenska, Ljubljana Bežigrad, Bežigrad",V večstanovanjski stavbi,...,NaN,1988.0,NaN,NaN,NaN,NaN,NaN,Zaprt balkon,NaN,NaN


In [10]:
data.columns


Index(['Unnamed: 0', 'Šifra oglasa', 'Objavljen ', 'Do poteka še',
       'Število prikazov oglasa', 'Cena', 'Vrsta cene', 'Posredovanje',
       'Lokacija', 'Umeščenost stanovanja', 'Nadstropje',
       'Skupno število nadstropij v stavbi', 'Vrsta stanovanja',
       'Bivalna površina', 'Neto površina', 'Leto izgradnje*',
       'Leto zadnje prenove', 'Opremljenost in stanje', 'Energetski razred',
       'Možnost menjave', 'Število parkirnih mest', 'Balkon / terasa / atrij',
       'Zunanje površine in oprema', 'Interna številka oglasa'],
      dtype='object')

In [20]:
data_feat = data.drop(columns=['Unnamed: 0', 'Šifra oglasa', 'Objavljen ', 'Do poteka še',
       'Število prikazov oglasa', 'Cena', 'Vrsta cene', 'Posredovanje','Interna številka oglasa','Možnost menjave', 'Zunanje površine in oprema'])
data_feat.head()

,Lokacija,Umeščenost stanovanja,Nadstropje,Skupno število nadstropij v stavbi,Vrsta stanovanja,Bivalna površina,Neto površina,Leto izgradnje*,Leto zadnje prenove,Opremljenost in stanje,Energetski razred,Število parkirnih mest,Balkon / terasa / atrij
0,"Savinjska, Celje, Slavko Šlander",V večstanovanjski stavbi,1.,2.0,3-sobno,"86,00 m²","105,00 m²",2004.0,NaN,NaN,NaN,NaN,NaN
1,"Osrednjeslovenska, Ljubljana Center, Center",V večstanovanjski stavbi,Pritličje,4.0,Garsonjera,"28,60 m²","29,30 m²",1908.0,2020.0,Popolnoma opremljeno,EI ni potrebna (334. člen EZ-1),NaN,NaN
2,"Gorenjska, Tržič, Tržič",V večstanovanjski stavbi,1.,5.0,Garsonjera,"11,00 m²","11,00 m²",1961.0,NaN,NaN,E (105 - 150 kWh/m2a),NaN,NaN
3,"Savinjska, Slovenske Konjice, Slovenske Konjice",V večstanovanjski stavbi,1.,5.0,2-sobno,NaN,"70,00 m²",2020.0,NaN,Neopremljeno,A2 (10 - 15 kWh/m2a),7+,"Balkon, Loža"
4,"Osrednjeslovenska, Ljubljana Bežigrad, Bežigrad",V večstanovanjski stavbi,7.,7.0,"1,5-sobno","53,50 m²",NaN,1988.0,NaN,NaN,NaN,NaN,Zaprt balkon


In [23]:
print(data_feat.shape)
print(data_feat.columns)

(1188, 13)
Index(['Lokacija', 'Umeščenost stanovanja', 'Nadstropje',
       'Skupno število nadstropij v stavbi', 'Vrsta stanovanja',
       'Bivalna površina', 'Neto površina', 'Leto izgradnje*',
       'Leto zadnje prenove', 'Opremljenost in stanje', 'Energetski razred',
       'Število parkirnih mest', 'Balkon / terasa / atrij'],
      dtype='object')
